In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tabulate import tabulate

In [62]:
#https://stackoverflow.com/questions/46135839/auto-detect-the-delimiter-in-a-csv-file-using-pd-read-csv
import csv

def get_delimiter(file_path, bytes = 4096):
    sniffer = csv.Sniffer()
    data = open(file_path, "r").read(bytes)
    delimiter = sniffer.sniff(data).delimiter
    return delimiter

In [63]:
def read_csv_file(file_path):
    try:
        for encoding in ['utf-8','latin1', 'ISO-8859-1']:
            delimiter= get_delimiter(file_path)
            try:
                df = pd.read_csv(file_path, low_memory=False, encoding=encoding, delimiter=delimiter)
                return df, True, None
            except (UnicodeDecodeError, pd.errors.ParserError) as e:
                pass  
        return None, False, f"Impossible de lire {file_path}."
    except FileNotFoundError as e:
        return None, False, e

In [64]:
years = list(range(2005, 2023, 1))
prefixes= ['caracteristiques', 'lieux', 'usagers', 'vehicules']
dataframes= []

for prefix in prefixes:
    datasets = []
    for year in years:
        connector = '_' if year <= 2016 else '-'
        file_name = f'data/raw/{prefix}{connector}{year}.csv'
        df, success, error = read_csv_file(file_name)
        if success:
            key= {file_name: df}
            datasets.append(key)
        else:
            print(f'{file_name} : {error}')
    dataframes.append(datasets)

print(f'Total datasets: {len(dataframes)}.')

for prefix, df_list in zip(prefixes, dataframes):
    print(f'{prefix}: {len(df_list)}.')

#Correction à la main des deux fichiers pour lesquels il y avait une erreur de frappe dans le nom.


Total datasets: 4.
caracteristiques: 18.
lieux: 18.
usagers: 18.
vehicules: 18.


In [65]:
#recuperer le nom et le type des colonnes
def get_df_structure(df):
    return tuple(sorted((col, df[col].dtype) for col in df.columns))


def group_dataframes_by_struct(dataframes):
    #regrouper les dataframes par groupe de structures similaires
    structure_groups = {}

    for df_list in dataframes:
        for item in df_list:
            for filename, df in item.items():
                structure = get_df_structure(df)
                if structure not in structure_groups:
                    structure_groups[structure] = []
                structure_groups[structure].append(filename)
    
        #regrouper les structures
    grouped_data = []
    for i, (structure, files) in enumerate(structure_groups.items(), start=1):
        unique_dtypes = set(dtype for _, dtype in structure)
        group_info = {
            "group_number": i,
            "num_columns": len(structure),
            "unique_dtypes": len(unique_dtypes),
            "structure": structure,
            "files": files
        }
        grouped_data.append(group_info)
        
    return grouped_data

In [66]:
# Afficher les groupes côte à côte / j'aurais pu le faire directement avec un dataframe
def display_grouped_data(grouped_data):
    table_headers = ["Groupe Numéro", "Nombre de Colonnes", "Nombre de Types Uniques", "Structure (Colonne:Type)", "Fichiers"]
    table_data = []

    for group in grouped_data:
        group_number = group['group_number']
        num_columns = group['num_columns']
        unique_dtypes = group['unique_dtypes']
        structure = group['structure']
        files = group['files']
        
        structure_str = "\n".join([f"{col}:{dtype}" for col, dtype in structure])
        files_str = "\n".join(files)
        
        row = [group_number, num_columns, unique_dtypes, structure_str, files_str]
        table_data.append(row)

    # Afficher le tableau
    print(tabulate(table_data, headers=table_headers, tablefmt="grid"))

In [67]:
#appliquer une structure à un dataframe
def apply_structure_to_df(df, structure):
    
    struct_cols = []
    for col, _ in structure:
        struct_cols.append(col)

    for col in df.columns:
        if col not in struct_cols:
            df.drop(col, axis= 1, inplace= True)
    
    for col, dtype in structure:
        if col not in df.columns:
            df[col] = pd.Series(dtype=dtype) 
        else:
            if pd.api.types.is_integer_dtype(dtype):
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
            elif pd.api.types.is_float_dtype(dtype):
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0).astype('float64')
            else:
                df[col] = df[col].astype(dtype) 
    
    df = df.reindex(columns= struct_cols)
    return df

In [68]:
#Choisir arbitrairement les structures à garder (les plus récentes)
#j'avais fais un système de séléction automatique mais c'est une très mauvaise idée...
target_df_caract= read_csv_file('data/raw/caracteristiques-2022.csv')[0]
target_df_caract.rename(columns= {'Accident_Id':'Num_Acc'}, inplace= True)

target_df_lieux= read_csv_file('data/raw/lieux-2022.csv')[0]
target_df_usagers= read_csv_file('data/raw/usagers-2022.csv')[0]
target_df_vehicules= read_csv_file('data/raw/vehicules-2022.csv')[0]

target_df_list = [get_df_structure(target_df_caract), get_df_structure(target_df_lieux), get_df_structure(target_df_usagers), get_df_structure(target_df_vehicules)]

#remplacer
filename, df = next(iter(dataframes[0][-1].items()))
df.rename(columns= {'Accident_Id':'Num_Acc'}, inplace= True)

#brutalement 
i= 0
for df_list in dataframes:
    for item in df_list:
        for filename, df in item.items():
            df = apply_structure_to_df(df, target_df_list[i])
    i+= 1



In [69]:
#grouper après traitement pour résultat
display_grouped_data(group_dataframes_by_struct(dataframes))

+-----------------+----------------------+---------------------------+----------------------------+------------------------------------+
|   Groupe Numéro |   Nombre de Colonnes |   Nombre de Types Uniques | Structure (Colonne:Type)   | Fichiers                           |
+=================+======================+===========================+============================+====================================+
|               1 |                   15 |                         2 | Num_Acc:int64              | data/raw/caracteristiques_2005.csv |
|                 |                      |                           | adr:object                 | data/raw/caracteristiques_2006.csv |
|                 |                      |                           | agg:int64                  | data/raw/caracteristiques_2007.csv |
|                 |                      |                           | an:int64                   | data/raw/caracteristiques_2008.csv |
|                 |                      

In [84]:
#on va tenter une concaténation sur tous les fichiers
full_datasets = []

for i in range(0, 4, 1):
    full_datasets.append(pd.DataFrame()) 

i= 0
for df_list in dataframes:
    for item in df_list:
        for filename, df in item.items():
            full_datasets[i] = pd.concat([df, full_datasets[i]])
    i+= 1

In [79]:
for df in full_datasets:
    display(df.head())
    display(df.tail())

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202200000001,19,10,2022,16:15,1,26,26198,2,3,1,3,TEIL(vieille route du),"44,5594200000","4,7257200000"
1,202200000002,20,10,2022,08:34,1,25,25204,2,3,1,3,Miranda,"46,9258100000","6,3462000000"
2,202200000003,20,10,2022,17:15,1,22,22360,2,6,1,2,ROND POINT DE BREZILLET,"48,4931620000","-2,7604390000"
3,202200000004,20,10,2022,18:00,1,16,16102,2,3,8,6,LOHMEYER (RUE),"45,6926520000","-0,3262900000"
4,202200000005,19,10,2022,11:45,1,13,13103,1,1,1,2,ROUTE DE JEAN MOULIN-RN 538,"43,6755790366","5,0927031775"


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
87021,200500087950,21,12,5,2035,5,974,416.0,2,2,1,3,"sans, LEBLOND(RUE M. ET",NaN,NaN
87022,200500087951,23,12,5,1010,1,974,416.0,2,1,1,3,"SANS, PRESIDENT MITTERAN",NaN,NaN
87023,200500087952,26,12,5,1715,1,974,416.0,2,2,1,3,"SANS, LEBLOND(RUE M. ET",NaN,NaN
87024,200500087953,27,12,5,1500,1,974,416.0,2,3,2,5,"SANS, HUBERT DE LISLE(BO",NaN,NaN
87025,200500087954,31,12,5,2100,5,974,414.0,2,1,2,7,RN01,NaN,NaN


,Num_Acc,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,202200000001,4,TEIL(vieille route du),0,NaN,2,2,0,1,(1),(1),1,NaN,-1,1,0,1,50.0
1,202200000002,4,NaN,0,NaN,2,2,0,1,(1),(1),1,NaN,-1,1,0,1,50.0
2,202200000003,3,ROND POINT DE BREZILLET,0,NaN,-1,2,0,1,0,0,1,NaN,-1,1,5,1,50.0
3,202200000004,4,QUATORZE JUILLET (RUE DU),0,NaN,1,1,0,2,(1),(1),1,NaN,4,1,0,1,30.0
4,202200000005,3,ROUTE DE JEAN MOULIN-RN 538,0,NaN,2,2,0,1,8,0,1,NaN,-1,1,0,1,80.0


,Num_Acc,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
87021,200500087950,4,0.0,0,NaN,2,3,0,1,NaN,NaN,1,0,80,1,0,1,NaN
87022,200500087951,4,0.0,0,NaN,2,2,0,1,NaN,NaN,2,0,70,1,0,1,NaN
87023,200500087952,4,0.0,0,NaN,1,2,0,1,NaN,NaN,1,0,75,1,0,1,NaN
87024,200500087953,4,0.0,0,NaN,2,0,0,1,NaN,NaN,1,0,150,1,0,1,NaN
87025,200500087954,2,0.0,0,NaN,3,2,0,1,76.0,0.0,2,20,75,2,0,1,NaN


,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202200000001,1 099 700,813 952,A01,1,1,3,1,2008.0,5,2.0,8.0,-1.0,-1,-1,-1
1,202200000001,1 099 701,813 953,B01,1,1,1,1,1948.0,5,1.0,8.0,-1.0,-1,-1,-1
2,202200000002,1 099 698,813 950,B01,1,1,4,1,1988.0,9,1.0,0.0,-1.0,0,0,-1
3,202200000002,1 099 699,813 951,A01,1,1,1,1,1970.0,4,1.0,0.0,-1.0,0,0,-1
4,202200000003,1 099 696,813 948,A01,1,1,1,1,2002.0,0,1.0,0.0,-1.0,-1,-1,-1


,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
197493,200500087953,NaN,NaN,B01,1,1,1,1,1972.0,0,NaN,NaN,NaN,0,0,0
197494,200500087953,NaN,NaN,C01,1,1,1,1,1965.0,0,NaN,NaN,NaN,0,0,0
197495,200500087953,NaN,NaN,A01,1,1,4,1,1990.0,5,NaN,NaN,NaN,0,0,0
197496,200500087954,NaN,NaN,A01,1,1,4,1,1951.0,0,NaN,NaN,NaN,0,0,0
197497,200500087954,NaN,NaN,A01,2,2,4,2,1946.0,0,NaN,NaN,NaN,0,0,0


,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202200000001,813 952,A01,1,2,0,2,1,9,1.0,0.0
1,202200000001,813 953,B01,1,7,0,2,2,1,1.0,0.0
2,202200000002,813 950,B01,2,7,0,2,8,15,1.0,0.0
3,202200000002,813 951,A01,2,10,0,2,1,1,1.0,0.0
4,202200000003,813 948,A01,2,7,0,2,1,2,1.0,0.0


,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
149759,200500087952,NaN,A01,0,7,0,2,3,1,NaN,0.0
149760,200500087953,NaN,B01,0,7,0,2,3,19,NaN,0.0
149761,200500087953,NaN,A01,0,2,0,2,1,17,NaN,0.0
149762,200500087953,NaN,C01,0,7,0,2,1,1,NaN,0.0
149763,200500087954,NaN,A01,0,7,4,0,0,1,NaN,0.0


In [83]:
merged_df = full_datasets[2]


for df_index, df in enumerate(full_datasets, start= 0):
    if df_index != 2:
        merged_df= pd.merge(merged_df, df, on= 'Num_Acc', how= 'left')

merged_df.set_index('id_usager', inplace= True)

KeyError: "None of ['id_usager'] are in the columns"

In [82]:
display(merged_df.head())
display(merged_df.tail())

,Num_Acc,id_vehicule_x,num_veh_x,place,catu,grav,sexe,an_nais,trajet,secu1,...,id_vehicule_y,num_veh_y,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202200000001,813 952,A01,1,1,3,1,2008.0,5,2.0,...,813 952,A01,1,2,0,2,1,9,1.0,0.0
1,202200000001,813 952,A01,1,1,3,1,2008.0,5,2.0,...,813 953,B01,1,7,0,2,2,1,1.0,0.0
2,202200000001,813 953,B01,1,1,1,1,1948.0,5,1.0,...,813 952,A01,1,2,0,2,1,9,1.0,0.0
3,202200000001,813 953,B01,1,1,1,1,1948.0,5,1.0,...,813 953,B01,1,7,0,2,2,1,1.0,0.0
4,202200000002,813 950,B01,1,1,4,1,1988.0,9,1.0,...,813 950,B01,2,7,0,2,8,15,1.0,0.0


,Num_Acc,id_vehicule_x,num_veh_x,place,catu,grav,sexe,an_nais,trajet,secu1,...,id_vehicule_y,num_veh_y,senc,catv,obs,obsm,choc,manv,motor,occutc
5004838,200500087953,NaN,A01,1,1,4,1,1990.0,5,NaN,...,NaN,B01,0,7,0,2,3,19,NaN,0.0
5004839,200500087953,NaN,A01,1,1,4,1,1990.0,5,NaN,...,NaN,A01,0,2,0,2,1,17,NaN,0.0
5004840,200500087953,NaN,A01,1,1,4,1,1990.0,5,NaN,...,NaN,C01,0,7,0,2,1,1,NaN,0.0
5004841,200500087954,NaN,A01,1,1,4,1,1951.0,0,NaN,...,NaN,A01,0,7,4,0,0,1,NaN,0.0
5004842,200500087954,NaN,A01,2,2,4,2,1946.0,0,NaN,...,NaN,A01,0,7,4,0,0,1,NaN,0.0
